In this notebook, we aim to perform the analysis leading to the plots in Figure3. 
This includes:
- Translation plots for each mouse region
- Scores (hit/near hit/miss) bar plot
- Pie charts on average scores per constituent part